In [1]:
import pandas as pd, numpy as np

import GEOparse, pyhgnc, pickle



from metabolitics.utils import load_network_model
from metabolitics.analysis import MetaboliticsAnalysis

from sklearn_utils.utils import SkUtilsIO
from sklearn_utils.preprocessing import FoldChangeScaler

from functools import reduce

In [2]:
class FoldChange:
    def __init__(self, fold_change):
        self.fold_change = fold_change

    # OR IS MAX, MAX IS PLUS, THUS OR IS PLUS
    def __add__(self, other):
        return max(self.fold_change, other.fold_change)
    
    # AND IS MIN, MIN IS MINUS, THUS AND IS MINUS
    def __sub__(self, other):
        return min(self.fold_change, other.fold_change)

In [3]:
class Genobolitics(MetaboliticsAnalysis):
    
    def set_objective(self, geo_utils):    
        self.clean_objective()
        for r in geo_utils.get_all_reactions():
            r.objective_coefficient = geo_utils.get_reaction_fold_change(r, nan_strategy=np.mean)
            

In [4]:
class GEOUtils:
    def __init__(self, geo_database_name, model, sample_name):
        self.database = GEOparse.get_GEO(geo=geo_database_name, destdir="./")\
                                .table.set_index("IDENTIFIER")[[sample_name]]
        self.database = self.clean_database()
        self.model = load_network_model(model)    
        self.genes = self.get_genes()
        self.reactions = self.get_all_reactions()
    
    def clean_database(self):
        ## TODO:Clean the database!
        ## TODO: handle NaNs more Hasanly
        return self.database.dropna()
    
    
    def get_genes(self):
        # use pyhgnc
        query = pyhgnc.query()
        genes_dict = dict()
        for symbol in self.database.index:
            try:
                genes_dict[symbol] = "HGNC:{}".format(query.hgnc(symbol=symbol)[0].identifier)
            except IndexError:
                continue
        return genes_dict.values()
        
    
    def get_reaction_fold_change(self, reaction):
        op = [('or', '+'), ('and', '-')]
        genes = [(gene, 'FoldChange({})'.format(
                self.get_gene_fold_change(gene))) for gene in self.get_reaction_genes(reaction)]
        
        return eval(reduce(lambda x, y: x.replace(*y), op + genes, reaction.gene_reaction_rule))
    
    def get_gene_fold_change(self, gene, nan_strategy=np.mean):
        return nan_stratgey(self.database.loc[gene])
    
    def get_all_reactions(self):
        return self.model.reactions
    
    def get_reaction_genes(self, reaction):
        return reaction.get_genes()